# Functions to build custom 2D histogram

In [ ]:
# Scale_Rectangle stretches an input rectangle to a new size
# Used to scale chips to a bigger size to represent on ROOT TCanvas
def scale_rectangle(bottom_left, top_right, center, alpha):
    # Calculate the new coordinates of bottom left corner
    new_bottom_left = (center[0] + alpha * (bottom_left[0] - center[0]), center[1] + alpha * (bottom_left[1] - center[1]))
    # Calculate the new coordinates of top right corner
    new_top_right = (center[0] + alpha * (top_right[0] - center[0]), center[1] + alpha * (top_right[1] - center[1]))

    return new_bottom_left + new_top_right

# MapColor maps an input acceptance value to a colour on a blue gradient colour scale
def MapColor(Peff):
    colors = [
    (204, 204, 255),  # Light blue
    (153, 178, 255),
    (102, 153, 255),
    (51, 128, 255),
    (0, 102, 255),
    (0, 77, 255),
    (0, 51, 204),
    (0, 26, 153),
    (0, 13, 128),
    (0, 0, 102)      # Dark blue
    ]
    x = Peff*100
    if x<=10:
        color = colors[0]
    elif x>10 and x<= 20:
        color = colors[1]
    elif x>20 and x<=30:
        color = colors[2]
    elif x>30 and x<=40:
        color = colors[3]
    elif x>40 and x<=50:
        color = colors[4]
    elif x>50 and x<=60:
        color = colors[5]
    elif x>60 and x<=70:
        color = colors[6]
    elif x>70 and x<=80:
        color = colors[7]
    elif x>80 and x<=90:
        color = colors[8]
    elif x>90:
        color = colors[9]
    else:
        color = (250,250,250)
        print(x)
    return (ROOT.TColor.GetColor(color[0],color[1],color[2]))       

# === The following functions return lists of ROOT objects, to be drawn on a ROOT TCanvas ===

# BuildBox uses the TBox class to build a box representing a chip.
# The box's colour represents the acceptance of the chip.
def BuildBox(Coordinates, border_colour, fill_bool, fill_colour=0, oppacity=1):
    if fill_bool:    
        Tbox = ROOT.TBox(Coordinates[0], Coordinates[1], Coordinates[2], Coordinates[3])
        Tbox.SetFillStyle(1001)
        Tbox.SetFillColor(fill_colour)
        Tbox.SetLineWidth(3)
        Tbox.SetLineColor(ROOT.kBlack)
        return Tbox
    else:
        Tbox = ROOT.TBox(Coordinates[0], Coordinates[1], Coordinates[2], Coordinates[3])
        Tbox.SetFillStyle(0)
        Tbox.SetLineColor(border_colour)
        Tbox.SetLineWidth(2)
        return Tbox

# BuildBoxes returns a list of TBox objects representing MFT chips.
# Makes use of the BuildBox function
def BuildBoxes(DF, Peff_DF, full_bool=False, par1=100, par2=25):
    # par1 and par2 are scaling parameters.
    # We draw larger chips for better visibility.
    DimChip = [1.3,3.0,0]
    ListID = Peff_DF['ChipID'].values
    nb_chip = np.size(ListID)

    Peff_array = Peff_DF['Pseaudo_Eff'].values
    min_Peff, max_Peff = np.min(Peff_array), np.max(Peff_array)
    
    # Building a list of all the boxes
    BoxeF0 = []
    BoxeF1 = []
    
    Colour_List = []
    idx = 0
    for i in ListID:
        half = DF.at[i+1,'Half']
        # Extracting the coordinates of the chip and translating them
        X, Y = GetTrueCoor(i+1, DF)[0] - DimChip[0], GetTrueCoor(i+1, DF)[1] - DimChip[1]

        bot_left = [(X+par2)/par1, (Y+par2)/par1]
        top_right = [(X+par2 + DimChip[0])/par1, (Y+par2 + DimChip[1])/par1]
        new_coord = scale_rectangle(bot_left, top_right, [0,0], 2) # Scaling the chip to make it bigger
        
        # Extracting colour
        Peff = Peff_DF.at[idx, 'Pseaudo_Eff']
        fill_colour = MapColor(Peff)
        
        # Extracting the layer
        layer = ExtractLayer(i)
        if layer%2 == 0:
            color = 2
            BoxeF0.append(BuildBox(new_coord, color, full_bool, fill_colour))
        else:
            color = 1
            BoxeF1.append(BuildBox(new_coord, color, full_bool, fill_colour))
        idx+=1
    return BoxeF0 + BoxeF1#, Colour_List

# Function to build the custom colour scale of the histogram
def BuildColorScale(Peff_DF, nb_division=10):
    Peff_array = Peff_DF['Pseaudo_Eff'].values
    min_Peff, max_Peff = np.min(Peff_array), np.max(Peff_array)    
    
    # Position of the scale:
    y_min, y_max = 0.1, 0.9
    x_min, x_max = 0.86, 0.91
    
    # Dimensions of the scale
    box_x_dim = x_max-x_min
    box_y_dim = (y_max-y_min)/nb_division
    
    # Position arrays of the elements of the Scale
    x_bot_left = np.array([x_min]*nb_division)
    y_bot_left = np.linspace(y_min,y_max-box_y_dim,nb_division)   
    x_top_right = x_bot_left + box_x_dim
    y_top_right = y_bot_left + box_y_dim    
    
    # Building scale contours
    text = ROOT.TText(x_min+(x_max-x_min)/2-0.05, y_max+0.04, 'Acceptance(%)')
    text.SetTextAlign(22)
    text.SetTextSize(0.035)
    text.SetTextColor(ROOT.kBlack)
    text.SetTextFont(1)
    
    line1 = ROOT.TLine(x_min, y_min, x_min, y_max)
    line1.SetLineColor(ROOT.kBlack)
    line1.SetLineWidth(1)

    line2 = ROOT.TLine(x_max, y_min, x_max, y_max)
    line2.SetLineColor(ROOT.kBlack)
    line2.SetLineWidth(1)

    List_Scale = [text, line1, line2] 
    
    # Filling Scale
    peffs = np.linspace(min_Peff,max_Peff,nb_division)
    dim_y = (y_max - y_min)/nb_division
    for i in range(0,nb_division):
        coor = [x_bot_left[i],y_bot_left[i], x_top_right[i], y_top_right[i]]
        List_Scale.append(BuildBox(coor, 2, True, MapColor((i+1)/10)))

        percent = (i+1)*10
        text = ROOT.TText(x_bot_left[i]+0.01 + box_x_dim+0.02, y_top_right[i], str(round(percent)))
        text.SetTextAlign(22)
        text.SetTextSize(0.03)
        text.SetTextColor(ROOT.kBlack)
        text.SetTextFont(1)
        List_Scale.append(text)
        
        line = ROOT.TLine(x_min-0.005, y_min+i*dim_y, x_max+0.005, y_min+i*dim_y)
        line.SetLineColor(ROOT.kBlack)
        line.SetLineWidth(1)
        List_Scale.append(line)
        
    line = ROOT.TLine(x_min-0.005, y_min+nb_division*dim_y, x_max+0.005, y_min+nb_division*dim_y)
    line.SetLineColor(ROOT.kBlack)
    line.SetLineWidth(1)
    List_Scale.append(line)

    return List_Scale

# Function to build the rest of the labels of the custom histogram (axis, titles,...)
def BuildGraphLabels(x_min = 0.86, y_min=0.1, x_max=0.91, y_max=0.9, nb_division = 20):    
    dim_y = (y_max - y_min)/nb_division
    l = []
    
    line0x = ROOT.TLine(0.49, 0.5, 0.51, 0.5)
    line0x.SetLineColor(ROOT.kBlack)
    line0x.SetLineWidth(1)
    l.append(line0x)
    
    line0y = ROOT.TLine(0.5, 0.49, 0.5, 0.51)
    line0y.SetLineColor(ROOT.kBlack)
    line0y.SetLineWidth(1)
    l.append(line0y)
    
    lineX = ROOT.TArrow(0.075, 0.1, 0.85, 0.1,0.05,">")
    l.append(lineX)
    
    lineY = ROOT.TArrow(0.15, 0.05, 0.15, 0.9,0.05, ">")
    l.append(lineY)
    
    negative_values = np.linspace(-15, 0, 5, endpoint=False)
    positive_values = np.linspace(0, 15, 5, endpoint=True)
    values = np.concatenate((negative_values, [0], positive_values))

    Xaxis = (values+par2)/50
    Yaxis = (values+par2)/50

    for i in range(0,np.size(Xaxis)):
        line = ROOT.TLine(Xaxis[i], 0.08, Xaxis[i], 0.12)
        line.SetLineColor(ROOT.kBlack)
        line.SetLineWidth(1)
        l.append(line)
        
        text = ROOT.TText(Xaxis[i], 0.06, str(values[i]))
        text.SetTextAlign(22)
        text.SetTextSize(0.025)
        text.SetTextColor(ROOT.kBlack)
        text.SetTextFont(1)
        l.append(text)
        
        line = ROOT.TLine(0.13, Yaxis[i], 0.17, Yaxis[i])
        line.SetLineColor(ROOT.kBlack)
        line.SetLineWidth(1)
        l.append(line)
                          
        text = ROOT.TText(0.11, Yaxis[i], str(values[i]))
        text.SetTextAlign(22)
        text.SetTextSize(0.025)
        text.SetTextColor(ROOT.kBlack)
        text.SetTextFont(1)
        l.append(text)
    
    XLabel = ROOT.TText(0.86, 0.035, "X (cm)")
    XLabel.SetTextAlign(22)
    XLabel.SetTextSize(0.035)
    XLabel.SetTextColor(ROOT.kBlack)
    XLabel.SetTextFont(1)
    l.append(XLabel)
    
    YLabel = ROOT.TText(0.035, 0.86, "Y (cm)")
    YLabel.SetTextAlign(11)
    YLabel.SetTextSize(0.035)
    YLabel.SetTextColor(ROOT.kBlack)
    YLabel.SetTextFont(1)
    l.append(YLabel)

    return l